In [2]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage


load_dotenv()

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")

llm = ChatOpenAI(
    model="deepseek/deepseek-v3.2",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    temperature=0.0
)


In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [4]:
system_prompt = """
You are a travel agent. NEVER ask follow-up questions.

ALWAYS use the **search-flight** tool (note the hyphen!) with these EXACT parameters:

- **flyFrom**: City or airport name/code (e.g., "San Francisco", "SFO")
- **flyTo**: City or airport name/code (e.g., "Tokyo", "HND")
- **departureDate**: Date in **dd/mm/yyyy** format (e.g., "31/03/2026")

Additional rules:
- If the user says "March 31st", convert it to "31/03/2026".
- Do NOT use ISO format (YYYY-MM-DD) — it will cause an error.
- You may omit optional parameters (like returnDate, passengers) if not specified.

After getting results, display them in a markdown table as described in the tool's instructions, and end with a fun fact about the destination.

NEVER make up flight data — always use the tool.
"""

In [12]:
agent = create_agent(
    model=llm,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt=system_prompt
)

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Get me a direct flight from Dubai to Phuket on March 31st")]},
    config
    )

print(response["messages"][-1].content)

I found several flights from Dubai to Phuket on March 31st, 2026. Let me organize them for you:

## **Cheapest Flights**

| Route | Departure & Arrival Times | Cabin Class | Price | Book |
|-------|---------------------------|-------------|-------|------|
| Dubai DXB → Mumbai BOM → Phuket HKT | 31/03 10:40 → 01/04 10:05 (20h 25m) | Economy | €213 | [Book](https://on.kiwi.com/epyyJi) |
| Dubai DXB → Mumbai BOM → Phuket HKT | 31/03 17:55 → 01/04 10:05 (13h 10m) | Economy | €213 | [Book](https://on.kiwi.com/5x2nKC) |
| Dubai DXB → Mumbai BOM → Phuket HKT | 31/03 22:05 → 01/04 10:05 (9h 0m) | Economy | €213 | [Book](https://on.kiwi.com/2jbbH5) |
| Dubai DXB → Chennai MAA → Phuket HKT | 31/03 19:40 → 01/04 08:05 (9h 25m) | Economy | €189 | [Book](https://on.kiwi.com/vLbriK) |

## **Shortest Flights**

| Route | Departure & Arrival Times | Cabin Class | Price | Book |
|-------|---------------------------|-------------|-------|------|
| Dubai DXB → Phuket HKT | 31/03 02:35 → 31/03 12:00 (6h 2